In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io.ascii import read
import fitsio

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
pypath = os.path.join(os.sep, 'global', 'work', 'projects', 'legacysurvey', 'legacycentrals')
meertcat = os.path.join(pypath, 'meert_et_al_data_tables_v2')
mendelcat = os.path.join(pypath, 'UPenn_PhotDec_Mstar_mlMendel14.dat')
wisccat = os.path.join(pypath, 'UPenn_PhotDec_Mstar_mlPCAWiscM11.dat')

In [3]:
rmpath = os.path.join(os.sep, 'global', 'work', 'projects', 'redmapper')
rmcatfile = os.path.join(rmpath, 'redmapper_isedfit_v5.10_centrals.fits.gz')

In [4]:
# Read the Mendel catalog
columns = ('GalCount', 'FlagSerExp', 'Mstar_Tab5_Pymorph',
           'Mstar_Tab5_Truncated', 'Mstar_Tab3_Pymorph',
           'Mstar_Tab3_Truncated', 'Mstar_Tab5_Mendel',
           'Mstar_Tab3_Mendel', 'Mstar_Tab5_cModel',
           'Mstar_Tab3_cModel')
dtype = np.dtype([(col, np.float) for col in columns])
allmendel = np.loadtxt(mendelcat, dtype=dtype)
#allmendel.dtype.names

In [5]:
# Keep good measurements and data.
keep = (np.where(allmendel['FlagSerExp'] == 0) and np.where(allmendel['Mstar_Tab5_Pymorph'] > 0))[0]
print('Keeping {} / {} measurements in the Mendel catalog.'.format(len(keep), len(allmendel)))
mendel = allmendel[keep]

Keeping 654801 / 670722 measurements in the Mendel catalog.


In [6]:
# Read the parent Meert catalog to get ra, dec and other info.
#upennpath = os.path.join(pypath, 'meert_et_al_data_tables_v2')
upenncatfile = os.path.join(meertcat, 'UPenn_PhotDec_CAST.fits')
upenncat = fitsio.read(upenncatfile, ext=1, rows=keep, upper=True)
#upenncat.dtype.names
len(upenncat)

654801

In [7]:
rminfo = fitsio.FITS(rmcatfile)
rmcat = rminfo[1].read(columns=['Z', 'RA', 'DEC', 'LAMBDA_CHISQ', 'MSTAR_50', 'MSTAR_ERR', 'IMAG', 'MAGGIES'])
print(rminfo[1])


  file: /global/work/projects/redmapper/redmapper_isedfit_v5.10_centrals.fits.gz
  extension: 1
  type: BINARY_TBL
  rows: 407118
  column info:
    MEM_MATCH_ID        i4  
    Z                   f4  
    RA                  f8  
    DEC                 f8  
    R                   f4  
    P                   f4  
    PFREE               f4  
    THETA_I             f4  
    THETA_R             f4  
    IMAG                f4  
    IMAG_ERR            f4  
    ZRED                f4  
    ZRED_E              f4  
    ZRED_CHISQ          f4  
    BCG_SPEC_Z          f4  
    Z_SPEC_INIT         f4  
    Z_INIT              f4  
    LAMBDA_CHISQ        f4  
    LAMBDA_CHISQ_E      f4  
    LAMBDA_ZRED         f4  
    LAMBDA_ZRED_E       f4  
    R_LAMBDA            f4  
    SCALEVAL            f4  
    MASKFRAC            f4  
    C_LAMBDA            f4  array[4]
    C_LAMBDA_ERR        f4  array[4]
    MAG_LAMBDA_ERR      f4  array[5]
    CHISQ               f4  
    Z_LAMBDA      

In [8]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [9]:
rcoord = SkyCoord(ra=rmcat['RA']*u.degree, dec=rmcat['DEC']*u.degree)
penncoord = SkyCoord(ra=upenncat['RA']*u.degree, dec=upenncat['DEC']*u.degree)
idx, sep2d, dist3d = rcoord.match_to_catalog_sky(penncoord, nthneighbor=1)

In [10]:
gd = np.where(sep2d < 3 * u.arcsec)[0]
len(gd)

23599

In [11]:
rmcat['MAGGIES']

array([[  8.08168750e-08,   6.74991668e-07,   1.72655052e-06, ...,
          3.43672355e-06,   3.23942504e-06,   1.95509529e-06],
       [  1.02759463e-08,   5.21811359e-08,   1.95987283e-07, ...,
          3.73946961e-07,   5.84767633e-07,   3.87884427e-07],
       [  6.44870024e-09,   1.60980221e-07,   4.88213459e-07, ...,
          8.71438601e-07,   1.20944264e-06,   7.12299254e-07],
       ..., 
       [  6.34218580e-11,   1.55727442e-09,   6.92663615e-09, ...,
          1.99032932e-08,   4.07453697e-08,   2.86100370e-08],
       [  1.79813442e-09,   1.88698146e-09,   4.91593877e-09, ...,
          7.67191821e-09,   1.36742511e-08,   1.15819407e-08],
       [  1.94307570e-09,   2.22601315e-09,   9.05395225e-09, ...,
          2.15409379e-08,   2.61627928e-08,   1.76007653e-08]], dtype=float32)